In [20]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
import dizionari

data = pd.read_csv("../data/ISTAT_Italian_Excursion_2014_2022.csv")


In [22]:
# Inizializzazione dell'app Dash con il tema Material
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MATERIA])


# Inizializzazione dei componenti di input
xaxis_feature_dropdown = dcc.Dropdown(
    id='xaxis-feature',
    options=[{'label': dizionari.dizionario_nomi[col], 'value': col} for col in data.columns],
    value='ID'  # Valore predefinito
)

yaxis_feature_dropdown = dcc.Dropdown(
    id='yaxis-feature',
    options=[{'label': dizionari.dizionario_nomi[col], 'value': col} for col in data.columns],
    value='mese'  # Valore predefinito
)

hist_feature_dropdown = dcc.Dropdown(
    id='hist-feature',
    options=[{'label': dizionari.dizionario_nomi[col], 'value': col} for col in data.columns],
    value='mese'  # Valore predefinito
)

violin_feature_dropdown = dcc.Dropdown(
    id='violin-feature',
    options=[{'label': dizionari.dizionario_nomi[col], 'value': col} for col in data.columns],
    value='mese'  # Valore predefinito
)

variabili_checklist = dbc.Checklist(
    id='correlation-checklist',
    options=[{'label': dizionari.dizionario_nomi[col], 'value': col} for col in data.columns],
    value=['mese', 'DEST_ERE', 'DEST_EIE'],  # Valori predefiniti
    inline=True
)

# Layout dell'app
app.layout = dbc.Container([
    html.H1("Analisi escursioni Italia"),
    dbc.Row([
        dbc.Col(
            dbc.Stack([
              html.Label("Seleziona la caratteristica dell'asse X:"),
              xaxis_feature_dropdown,
              html.Label("Seleziona la caratteristica dell'asse Y:"),
              yaxis_feature_dropdown,
              ])
            , width=6),
        dbc.Col(
            dbc.Stack([
              html.Label("Seleziona la caratteristica dell'asse Y:"),
              hist_feature_dropdown,
            ])
        , width=6, align="center"),
    ]),
    dbc.Row([
        dbc.Col(
            dbc.Stack([
              dcc.Graph(id='graph')
              ])
            , width=6),
        dbc.Col(
            dbc.Stack([
              dcc.Graph(id='histogram')
            ])
        , width=6),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Stack([
                html.Label("Select a feature to visualize with a violin plot:"),
                violin_feature_dropdown,
            ])
        ], width=4, align="center"),
        dbc.Col([
            dbc.Stack([
              html.Label("Seleziona le variabili per la heatmap di correlazione:"),
              variabili_checklist,
            ])
        ], width=8, align="center"),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Stack([
                dcc.Graph(id='violin-plot')
            ])
        ], width=4),
        dbc.Col([
            dbc.Stack([
              dcc.Graph(id='heatmap')
            ])
        ], width=8),
    ]),
])

# LINECHART - Evoluzione degli spostami negli anni (suddivisi per età)
# Callback per il grafico a dispersione (scatter plot)
@app.callback(
    Output('graph', 'figure'),
    [Input('xaxis-feature', 'value'),
     Input('yaxis-feature', 'value')]
)
def update_line_chart(xaxis_feature, yaxis_feature):
  data['Età'] = data['eta10'].map(dizionari.eta_mapping)
  conteggio = data.groupby(['annrif', 'Età']).size().reset_index(name='count')
  print(conteggio)
  # Colori che vanno dal rosso (<= 14 yo) al verde (>= 75 yo)
  colori = ['#FF0000', '#FF4500', '#FF8C00', '#FFD700', '#ADFF2F', '#7FFF00', '#32CD32', '#008000']

  fig = px.line(conteggio,
                x='annrif',
                y='count',
                color="Età",
                title='Evoluzione degli spostamenti negli anni (Età)',
                color_discrete_sequence=colori)
  
  fig.update_layout(margin=dict(l=50, r=50, t=50, b=50))
  
  # fig = px.scatter(boston_data, x=xaxis_feature, y=yaxis_feature, color='mese',
  #                    title=f'Grafico a dispersione di {dizionari.dizionario_nomi[yaxis_feature]} vs {dizionari.dizionario_nomi[xaxis_feature]}',
  #                    labels={xaxis_feature: xaxis_feature, yaxis_feature: yaxis_feature})
    
  return fig

# Esecuzione dell'app
if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="external")





Dash app running on http://127.0.0.1:8050/


    annrif    Età  count
0     2014   <=14    708
1     2014  15-24    356
2     2014  25-34    482
3     2014  35-44    855
4     2014  45-54    854
..     ...    ...    ...
67    2022  35-44    611
68    2022  45-54    787
69    2022  55-64    892
70    2022  65-74    605
71    2022   >=75    223

[72 rows x 3 columns]
    annrif    Età  count
0     2014   <=14    708
1     2014  15-24    356
2     2014  25-34    482
3     2014  35-44    855
4     2014  45-54    854
..     ...    ...    ...
67    2022  35-44    611
68    2022  45-54    787
69    2022  55-64    892
70    2022  65-74    605
71    2022   >=75    223

[72 rows x 3 columns]
    annrif    Età  count
0     2014   <=14    708
1     2014  15-24    356
2     2014  25-34    482
3     2014  35-44    855
4     2014  45-54    854
..     ...    ...    ...
67    2022  35-44    611
68    2022  45-54    787
69    2022  55-64    892
70    2022  65-74    605
71    2022   >=75    223

[72 rows x 3 columns]
